In [1]:
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import datasets

Using TensorFlow backend.


In [2]:
num_epoch = 5
batch_size = 128

In [3]:
# 加载数据集
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [4]:
# min-max归一化
X_train = X_train / 255
X_test = X_test / 255

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

num_examples = X_train.shape[0]
total_batch = int(num_examples/batch_size)

In [5]:
_x = tf.placeholder(dtype=tf.float32, shape=(None, 784))
_y = tf.placeholder(dtype=tf.float32, shape=(None, 10))
# 三个全连接层
L1 = tf.layers.dense(units=512, activation=tf.nn.relu, inputs=_x)
L2 = tf.layers.dense(units=256, activation=tf.nn.relu, inputs=L1)
L3 = tf.layers.dense(units=10, activation=tf.nn.softmax, inputs=L2)
print(L3)

Tensor("dense_2/Softmax:0", shape=(?, 10), dtype=float32)


In [6]:
# 损失函数：交叉熵
loss_op = - tf.reduce_mean(_y * tf.log(tf.clip_by_value(L3,1e-10, 1)))
train_op = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(loss_op)
init_op = tf.global_variables_initializer()

# 正确预测
correct_predict = tf.equal(tf.argmax(L3, axis=1), tf.argmax(_y, axis=1))
# 准确率
acc_op = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
print(loss_op)

Tensor("Neg:0", shape=(), dtype=float32)


In [7]:
def next_batch(i):
    a = i * batch_size
    b = (i+1) * batch_size
    return (X_train[a:b], y_train[a:b])

In [8]:
with tf.Session() as sess:
    sess.run(init_op)
    
    for epoch in range(num_epoch):
        avg_cost = 0.
        avg_acc = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(i)
            sess.run(train_op, feed_dict={_x: batch_xs, _y: batch_ys})
            c, acc = sess.run([loss_op, acc_op], feed_dict={_x: batch_xs, _y: batch_ys})
            avg_cost += c / total_batch
            avg_acc += acc / total_batch
        c1, acc1 = sess.run([loss_op, acc_op], feed_dict={_x: X_test, _y: y_test})
        print("Epoch:", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_cost), "accuracy=", "{:.9f}".format(avg_acc), 
              "test_loss=", "{:.9f}".format(c1), "test_accuracy=", "{:.9f}".format(acc1))
    print("Optimization Finished!")

Epoch: 0001 loss= 0.027794102 accuracy= 0.924896501 test_loss= 0.014853272 test_accuracy= 0.953899980
Epoch: 0002 loss= 0.010468065 accuracy= 0.970035390 test_loss= 0.010972422 test_accuracy= 0.964100003
Epoch: 0003 loss= 0.006396540 accuracy= 0.981937767 test_loss= 0.008936489 test_accuracy= 0.970700026
Epoch: 0004 loss= 0.004161438 accuracy= 0.989349626 test_loss= 0.007910130 test_accuracy= 0.975700021
Epoch: 0005 loss= 0.002743818 accuracy= 0.993706597 test_loss= 0.007949500 test_accuracy= 0.976300001
Optimization Finished!
